In [ ]:
#Download CIFAR 10 dataset for training and validation purposes and apply the following changes on each image:
# 1) make it a tensor
# 2) normalize it based on the mean and standard deviation among all pixels in each channel (RGB).
#Print the size of training and validation datasets

In [ ]:
#We want to make a tertiary classifier that distinguishes between deers, dogs, and horses, labeled as 4, 5, and 7, resp.
#Create the subset training and validation datasets for this purpose.
#Print the size of these datasets.

In [ ]:
#Create a parameterized CNN with the following details. 
# The parameter is the number of output channels n after the first convolution.
# All kernels are of size 3 by 3.
# Convolutions must not change the height and width.
# Each convolution is followed by hyperbolic tangent as the activation function, and max pooling of size 2 by 2.
# Convolution ayers:
# 1) First convolution layer works on the input RGB input. Let's assume there are n kernels in this layer.
# 2) Second convolution layer works on the result of the preceding max pooling layer. 
#    Let's assume there are n/2 kernels in this layer.
# 3) Third convolution layer works on the result of the preceding max pooling layer. 
#    Let's assume there are n/2 kernels in this layer. 
# Fully connected layers:
# 1) First fully connected layer works on the result of the preceding max pooling layer. 
#    This layer is followed by hyperbolic tangent as its activation function.
# 2) Second fully connected layer works on the result of the preceding activation function, and emits numbers associated
#    with each class.
# We will use negative log likelihood to compute the loss. So you may add additional layer(s) to your network.
# Note: Since the network is parameterized (n), you'd rather define the CNN as a subclass of nn.Module.

In [ ]:
#Create two networks as instances of the CNN you defined above, with n = 16 and n = 32 respectively. 
#Print the total number of parameters in each of these instances.

In [ ]:
#Our training functionality is supposed to compute gradient on batches of training data, randlomy selected each time.
#To this end, create a training data loader with batch size 32 that randomizes access to each batch.
#Also, create a validation data loader with the same batch size that does not randomize access to each batch (no need!)
#Print the number of batches in training and validation data loaders

In [ ]:
#Define your training function that receives the training loader, model, loss function, optimizer, the device (cpu/gpu), and 
# number of epochs.
#In each epoch, you should go through each training data batch, and:
# 1) move data to device
# 1) compute the output batch, and accordingly the loss
# 2) compute the gradient of loss wrt parameters, and update the parameters
#After covering all epochs, your training function must report the training accuracy

In [ ]:
#Define a separate function that receives the validation data loader as well as the model and computes the validation 
# accuracy of the model.

In [ ]:
#Define device dynamically based on whether CUDA is available or not.
#Call the training function on the created training data loader, the created CNN  with n = 16, 
# negative log likelihood loss function, stochastic gradient descent optimizer,
# the device you defined, and 100 epochs. Next, call validation accuracy function.
#Is the model overfit? (Yes/No) Why?

#Define the device (CPU in the absence of a GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#16-node model training and validation.
model_n16 = CNN(16)  # Create a 16-n CNN model
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.SGD(model_n16.parameters(), lr=learning_rate)  # Optimizer

train(trainloader, model_n16, criterion, optimizer, device, epochs) # Update parameters, calculate loss, and train the model
validate(testloader, model_n16, device) # Utilize data validation to validate the training model

In [ ]:
#Call the training function on the created training data loader, the created CNN  with n = 32, 
# negative log likelihood loss function, stochastic gradient descent optimizer,
# the device you defined, and 100 epochs. Next, call validation accuracy function.
#Is the model overfit? (Yes/No) Why? 
# (This can be compared to the fully connected network we created in the last set of exercises.)

model_n32 = CNN(32) # Make a 32-n CNN model
optimizer = optim.SGD(model_n32.parameters(), lr=learning_rate)  # Optimizer
train_model(trainloader, model_n32, criterion, optimizer, device, epochs) # Update parameters, calculate loss, and train the model
validate_model(testloader, model_n32, device) # Utilize data validation to validate the training model

In [ ]:
#Next, let's consider L2 regularization with weight decay 0.002 for CNN with n = 32. 
# Is the model overfit? (Yes/No) Why?

weight_decay = 0.002 # Decrease in weight in L2 regularization
optimizer_l2 = optim.SGD(model_n32.parameters(), lr=learning_rate, weight_decay=weight_decay)  # an L2 regularized optimizer
train_model(trainloader, model_n32, criterion, optimizer_l2, device, epochs) # Update parameters, calculate loss, and train the model
validate_model(testloader, model_n32, device) # Utilize data validation to validate the training model

In [ ]:
#Add a skip connection in your CNN from the output of second max pooling to the input of 3rd max pooling.
#Train the updated CNN with the same parameters including (n = 32).
#Is the model overfit? (Yes/No) Why?

class SkipCNN(nn.Module):
    # skip connections modified CNN model
    def __init__(self, n):
        super(SkipeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, n, 3) # 3 input channels, n output channels, and a 3x3 kernel are used in the convolutional layer
        self.conv2 = nn.Conv2d(n, n//2, 3) # n input channels, n//2 output channels, and a 3x3 kernel are used in the convolutional layer
        self.conv3 = nn.Conv2d(n//2, n//2, 3) # n//2 input channels, n//2 output channels, and a 3x3 kernel are used in the convolutional layer
        self.fc1 = nn.Linear(n//2 * 2 * 2, n//2) # n//2 * 2 * 2 feature mappings to n//2 neurons are flattened by the first connected layer
        self.fc2 = nn.Linear(n//2, len(class_indices)) # n//2 input neurons, output neurons based on class indices
        self.activation = nn.Tanh() # Hyperbolic Tangent is the activation function (Tanh)
        self.pool = nn.MaxPool2d(2, 2) # 2x2 kernel, stride 2, maximum pooling layer
    def forward(self, x):
        x1 = self.pool(self.activation(self.conv1(x))) # Convolution, activation, and pooling are applied to the first layer
        x2 = self.pool(self.activation(self.conv2(x1))) # Convolution, activation, and pooling are applied to the second layer
        x3 = self.pool(self.activation(self.conv3(x2))) # Convolution, activation, and pooling are applied to the third layer
        x = x3.view(-1, self.num_flat_features(x3)) # Tensor flattening for connected layers
        x = self.activation(self.fc1(x)) # activating the top layer that is connected
        x = self.fc2(x) # Applied a second connected layer.
        return x
    def num_flat_features(self, x): # Total the features in a tensor without using the batch dimension
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
model_skip = SkipCNN(32) # Model a 32-n SkipCNN
optimizer_skip = optim.SGD(model_skip.parameters(), lr=learning_rate) # Optimizer
train_model(trainloader, model_skip, criterion, optimizer_skip, device, epochs) # Update parameters, calculate loss, and train the model
validate_model(testloader, model_skip, device) #  Utilize data validation to validate the training model

In [ ]:
#Consider dropout layers after each max pooling in the original CNN, where the probability of zeroing output features is 30%.
#Train the updated CNN with the same parameters including (n = 32).
#Is the model overfit? (Yes/No) Why?

#From nn.Module, create the DropoutCNN neural network class.
class DropoutCNN(nn.Module):
    # The 'n' input in the class constructor denotes the number of channels in a convolutional layer.
    def __init__(self, n):
        super(DropoutCNN, self).__init__() 
        self.conv1 = nn.Conv2d(3, n, 3) # 3 input channels, 'n' output channels, and 3 kernel sizes make up the first convolutional layer
        self.conv2 = nn.Conv2d(n, n//2, 3) # 'n' input channels, 'n//2' output channles, and 3 kernel sizes make up the second convolutional layer
        self.conv3 = nn.Conv2d(n//2, n//2, 3) # 'n//2' input channles, 'n//2' output channels, and 3 kernel sizes make up the third convulutional layer
        self.fc1 = nn.Linear(n//2 * 2 * 2, n//2) # With input characteristics of "n//2 * 2 * 2" and output features of "n//2," the first connected linear layer
        self.fc2 = nn.Linear(n//2, len(class_indices)) # Create a linear layer with n//2 input features and class_indices length output features
        self.activation = nn.Tanh() # Make the activation function's initial value a hyperbolic tangent (Tanh)
        self.pool = nn.MaxPool2d(2, 2) # Make a maximum 2x2 pooling layer
        self.dropout = nn.Dropout(p=0.3) # Create a layer with a 0.3% dropout rate

    # Network forward pass is defined by the forward method
    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x))) # Applying the first convolutional layer, then activating and max pooling
        x = self.dropout(x) # To the output, apply dropout
        x = self.pool(self.activation(self.conv2(x))) # Applying the second convolutional layer, then activating and max pooling
        x = self.dropout(x) # To the output, apply dropout
        x = self.pool(self.activation(self.conv3(x))) # Applying the third convolutional layer, then activating and max pooling
        x = x.view(-1, self.num_flat_features(x)) # Connected layer shape tensor
        x = self.activation(self.fc1(x)) # Activate the output of the first connected layer
        x = self.fc2(x) # apply the connected second layer
        return x # Return the final tensor
    
    def num_flat_features(self, x): # Total the features in a tensor without using the batch dimension
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model_dropout = DropoutCNN(32) # Create a 32-n DropoutCNN model.
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=learning_rate) # Optimizer
train_model(trainloader, model_dropout, criterion, optimizer_dropout, device, epochs) # Update parameters, calculate loss, and train the model
validate_model(testloader, model_dropout, device) #  Utilize data validation to validate the training model

In [ ]:
#Considering all the modifications which one works better? Plain CNN, CNN+L2, CNN+Skip, CNN+Dropout?